In [ ]:
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importing dataset is stored locally
df=pd.read_csv('/content/drive/MyDrive/mbti_1.csv', dtype={'type': str, 'posts': str})
#df.info()
mbti = {'I':'Introversion', 'E':'Extroversion', 'N':'Intuition',
        'S':'Sensing', 'T':'Thinking', 'F': 'Feeling',
        'J':'Judging', 'P': 'Perceiving'}

#TODO: Raus, nur drin um erstmal Zeit zu sparen
df = df.head(200)

In [ ]:
df_counted = df['type'].value_counts()
df_counted.head()

plt.figure(figsize=(12,4))
sns.barplot(x = df_counted.index, y = df_counted.values)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Types', fontsize=12)
plt.show()

In [ ]:
#Split an ||| 
df['posts_processed']=df['posts'].apply(lambda x: x.split('|||'))
df.head()

In [ ]:
#Alles zu lower case
for posts in df['posts_processed'].items() :
    for post in posts[1] : 
      post = post.lower();

df.head()

In [ ]:
def process_list(posts):
  result = []
  for post in posts : 
      result.append(post.lower())
      #Von Caro für das abkürzen von loooooove zu loove
      #post = ''.join(''.join(s)[:2] for _, s in itertools.groupby(post))
  return result

df['posts_processed2']=df['posts_processed'].apply(process_list)

df.head()

In [ ]:

# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

# Expanding Contractions in the reviews
df['posts_processed']=df['posts_processed'].apply(lambda x : expand_contractions(x))

In [ ]:
def dequote(s):
    """
    If a string has single or double quotes around it, remove them.
    Make sure the pair of quotes match.
    If a matching pair of quotes is not found, return the string unchanged.
    """
    if (s[0] == s[-1]) and s.startswith(("'", '"')):
        return s[1:-1]
    return s

#Anführungszeichen entfernen vorne und hinten
df['posts_processed'] = df['posts_processed'].apply(dequote)

In [ ]:
#Links in den Kommentaren Zählen
df['links_per_post'] = df['posts_processed'].apply(lambda x: x.count('http')/50)
print('Count Links')
df.head(5)

In [ ]:
def remove_URL(stringliteral):
    """Remove URLs from a sample string"""
    return re.sub(r'https?://\S+', '', str(stringliteral))

df['posts_processed']=df['posts_processed'].map(remove_URL)
print('Removed Links')
df.head(5)

In [ ]:
#Satzzeichen entfernen
df['posts_processed']=df['posts_processed'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
print(df.at[0,'posts_processed'])

In [ ]:
#Zahlen und Wörter mit Zahlen entfernen
df['posts_processed']=df['posts_processed'].apply(lambda x: re.sub('\w*\d\w*','', x))
print(df.at[0,'posts_processed'])

In [ ]:
# Removing extra spaces
df['posts_processed']=df['posts_processed'].apply(lambda x: re.sub(' +',' ',x))
print('Compare Processing')
print('Problem beim Splitten ||| da das wort davor und danach als ein einziges wort betrachtet werden!')
print(df.at[0,'posts'])
print(df.at[0,'posts_processed'])

In [ ]:
#Durchschnittl. Wörter per Post
df['avg_words_per_post'] = df['posts_processed'].apply(lambda x: len(x.split())/50)
print('Avg Words per Post')
df.head(5)

In [ ]:
plt.figure(figsize=(15,10))
sns.swarmplot("type", "avg_words_per_post", data=df)

In [ ]:
#Anzahl der Links anzeigen
plt.figure(figsize=(15,10))
sns.swarmplot("type", "links_per_post", data=df)
